### For the machine learning approach I will start with a binary classification using XGBoost classifier  

### So bacially we're deciding "will this fither become a top tier contender, yes or no?"

In [6]:
import pandas as pd

In [5]:
df_combined = pd.read_csv("Data/combined_ufc_dataset.csv")

c:\Users\emilp\anaconda3\envs\gputest\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df_combined

,EVENT,BOUT,ROUND,FIGHTER,KD,SIGSTR_,TOTAL_STR,TD_,SUBATT,REV,...,FINISHDETAILS,FINISHROUND,TOTALFIGHTTIMESECS,REDDECODDS,BLUEDECODDS,RSUBODDS,BSUBODDS,RKOODDS,BKOODDS,FINISHROUNDTIME_SECS
0,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,1,alex pereira,0.0,47%,20 of 38,---,0.0,0.0,...,Punch,4.0,1172.0,900.000000,425.870288,1000.000000,1100.497775,-295.000000,636.463235,272.0
1,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,1,alex pereira,0.0,47%,20 of 38,---,0.0,0.0,...,Kick,2.0,313.0,650.000000,900.000000,1800.000000,1000.000000,110.000000,180.000000,13.0
2,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,1,alex pereira,0.0,47%,20 of 38,---,0.0,0.0,...,Punch,1.0,194.0,600.000000,650.000000,2000.000000,1400.000000,110.000000,200.000000,194.0
3,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,1,alex pereira,0.0,47%,20 of 38,---,0.0,0.0,...,Punch,2.0,561.0,308.333395,425.870288,884.048151,1100.497775,510.891606,636.463235,261.0
4,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,1,alex pereira,0.0,47%,20 of 38,---,0.0,0.0,...,Unknown,3.0,900.0,800.000000,900.000000,1600.000000,1600.000000,-135.000000,225.000000,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392712,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,1,david levicki,0.0,80%,95 of 102,---,0.0,0.0,...,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
392713,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,1,patrick smith,0.0,100%,1 of 1,0%,1.0,0.0,...,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
392714,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,1,ray wizard,0.0,100%,2 of 2,---,0.0,0.0,...,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
392715,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,1,scott morris,0.0,100%,2 of 2,100%,1.0,0.0,...,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [6]:
df_combined['KD_RATE'] = df_combined['KD'] / df_combined['ROUND'].replace(0, 1)
df_combined['SIGSTR_ACC'] = df_combined['SIGSTR_MADE'] / df_combined['SIGSTR_ATTEMPTED'].replace(0, 1)
df_combined['TD_ACC'] = df_combined['TD_MADE'] / df_combined['TD_ATTEMPTED'].replace(0, 1)
df_combined['FINISH_RATIO'] = (df_combined['SUBATT'] + df_combined['KD']) / df_combined['ROUND'].replace(0, 1)

In [11]:
selected_features = [
    'KD_RATE',
    'SIGSTR_ACC',
    'TD_ACC',
    'FINISH_RATIO',
    'CTRL',
    'REV',
    'REACHDIF',
    'HEIGHTDIF',
    'AGEDIF',
    'REDWINS',
    'REDLOSSES',
    'REDCURRENTWINSTREAK',
    'REDLONGESTWINSTREAK',
    'REDTOTALTITLEBOUTS'
]

In [12]:
# Example: Label fighters with more than 10 wins as "top-tier"
df_combined['LABEL'] = (df_combined['REDWINS'] >= 10).astype(int)

In [ ]:
print(df_combined.columns.tolist())

['EVENT', 'BOUT', 'ROUND', 'FIGHTER', 'KD', 'SIGSTR_', 'TOTAL_STR', 'TD_', 'SUBATT', 'REV', 'CTRL', 'SIGSTR_MADE', 'SIGSTR_ATTEMPTED', 'HEAD_MADE', 'HEAD_ATTEMPTED', 'BODY_MADE', 'BODY_ATTEMPTED', 'LEG_MADE', 'LEG_ATTEMPTED', 'DISTANCE_MADE', 'DISTANCE_ATTEMPTED', 'CLINCH_MADE', 'CLINCH_ATTEMPTED', 'GROUND_MADE', 'GROUND_ATTEMPTED', 'TD_MADE', 'TD_ATTEMPTED', 'REDFIGHTER', 'BLUEFIGHTER', 'REDODDS', 'BLUEODDS', 'REDEXPECTEDVALUE', 'BLUEEXPECTEDVALUE', 'DATE', 'LOCATION', 'COUNTRY', 'WINNER', 'TITLEBOUT', 'WEIGHTCLASS', 'GENDER', 'NUMBEROFROUNDS', 'BLUECURRENTLOSESTREAK', 'BLUECURRENTWINSTREAK', 'BLUEDRAWS', 'BLUEAVGSIGSTRLANDED', 'BLUEAVGSIGSTRPCT', 'BLUEAVGSUBATT', 'BLUEAVGTDLANDED', 'BLUEAVGTDPCT', 'BLUELONGESTWINSTREAK', 'BLUELOSSES', 'BLUETOTALROUNDSFOUGHT', 'BLUETOTALTITLEBOUTS', 'BLUEWINSBYDECISIONMAJORITY', 'BLUEWINSBYDECISIONSPLIT', 'BLUEWINSBYDECISIONUNANIMOUS', 'BLUEWINSBYKO', 'BLUEWINSBYSUBMISSION', 'BLUEWINSBYTKODOCTORSTOPPAGE', 'BLUEWINS', 'BLUESTANCE', 'BLUEHEIGHTCMS', 'BL

In [13]:
from sklearn.model_selection import train_test_split

X = df_combined[selected_features]
y = df_combined['LABEL']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from xgboost import XGBClassifier

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [18]:
print("Training size:", X_train.shape)

Training size: (314173, 14)


In [17]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     60755
           1       1.00      1.00      1.00     17789

    accuracy                           1.00     78544
   macro avg       1.00      1.00      1.00     78544
weighted avg       1.00      1.00      1.00     78544

[[60755     0]
 [    0 17789]]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

importances = model.feature_importances_
feature_df = pd.DataFrame({'feature': selected_features, 'importance': importances})
feature_df = feature_df.sort_values('importance', ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(data=feature_df, x='importance', y='feature')
plt.title("Feature Importance (XGBoost)")
plt.tight_layout()
plt.show()

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier

# Load dataset
df_combined = pd.read_csv("Data/combined_ufc_dataset.csv")

# Create new features
df_combined['KD_RATE'] = df_combined['KD'] / df_combined['ROUND'].replace(0, 1)
df_combined['SIGSTR_ACC'] = df_combined['SIGSTR_MADE'] / df_combined['SIGSTR_ATTEMPTED'].replace(0, 1)
df_combined['TD_ACC'] = df_combined['TD_MADE'] / df_combined['TD_ATTEMPTED'].replace(0, 1)
df_combined['FINISH_RATIO'] = (df_combined['SUBATT'] + df_combined['KD']) / df_combined['ROUND'].replace(0, 1)

# Label: top-tier fighters (10+ wins)
df_combined['LABEL'] = (df_combined['REDWINS'] >= 10).astype(int)

# Define features — NOTE: REDWINS is removed to prevent leakage
selected_features = [
    'KD_RATE',
    'SIGSTR_ACC',
    'TD_ACC',
    'FINISH_RATIO',
    'CTRL',
    'REV',
    'REACHDIF',
    'HEIGHTDIF',
    'AGEDIF'
]


X = df_combined[selected_features]
y = df_combined['LABEL']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))


c:\Users\emilp\anaconda3\envs\gputest\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


              precision    recall  f1-score   support

           0       0.82      0.98      0.90     60755
           1       0.84      0.28      0.42     17789

    accuracy                           0.82     78544
   macro avg       0.83      0.63      0.66     78544
weighted avg       0.83      0.82      0.79     78544

[[59800   955]
 [12877  4912]]


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Load dataset
df_combined = pd.read_csv("Data/combined_ufc_dataset.csv")

# Step 1: Handle NaN values in 'REDWINS' column (fill NaN with 0)
df_combined['REDWINS'] = df_combined['REDWINS'].fillna(0)

# Step 2: Create new features
df_combined['KD_RATE'] = df_combined['KD'] / df_combined['ROUND'].replace(0, 1)
df_combined['SIGSTR_ACC'] = df_combined['SIGSTR_MADE'] / df_combined['SIGSTR_ATTEMPTED'].replace(0, 1)
df_combined['TD_ACC'] = df_combined['TD_MADE'] / df_combined['TD_ATTEMPTED'].replace(0, 1)
df_combined['FINISH_RATIO'] = (df_combined['SUBATT'] + df_combined['KD']) / df_combined['ROUND'].replace(0, 1)

# Step 3: Define target variable as multiclass (3 classes here: Top-tier, Mid-tier, Low-tier)
# Ensure there are no NaN or invalid values before applying pd.cut
df_combined['REDWINS'] = pd.to_numeric(df_combined['REDWINS'], errors='coerce')
df_combined['LABEL'] = pd.cut(df_combined['REDWINS'], bins=[0, 5, 10, 50], labels=['Low-tier', 'Mid-tier', 'Top-tier'], right=False)

# Ensure that 'LABEL' is a categorical variable
df_combined['LABEL'] = df_combined['LABEL'].astype('category')

# Step 4: Encode the LABEL column to numerical values
label_encoder = LabelEncoder()
df_combined['LABEL'] = label_encoder.fit_transform(df_combined['LABEL'])

# Step 5: Define features — NOTE: REDWINS is removed to prevent leakage
selected_features = [
    'KD_RATE',
    'SIGSTR_ACC',
    'TD_ACC',
    'FINISH_RATIO',
    'CTRL',
    'REV',
    'REACHDIF',
    'HEIGHTDIF',
    'AGEDIF'
]

X = df_combined[selected_features]
y = df_combined['LABEL']

# Step 6: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Set up the XGBoost model for multiclass classification
model = XGBClassifier(
    objective='multi:softmax',  # Specify multiclass classification
    num_class=3,  # Number of classes (adjust based on your target variable)
    use_label_encoder=False,
    eval_metric='mlogloss'
)

# Step 8: Train the model
model.fit(X_train, y_train)

# Step 9: Make predictions
y_pred = model.predict(X_test)

# Step 10: Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.83      0.69     35913
           1       0.62      0.41      0.49     24842
           2       0.66      0.42      0.51     17789

    accuracy                           0.61     78544
   macro avg       0.62      0.55      0.57     78544
weighted avg       0.62      0.61      0.59     78544

Confusion Matrix:
[[29979  3939  1995]
 [12791 10168  1883]
 [ 8095  2246  7448]]


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Load the main dataset (combined UFC dataset)
df_combined = pd.read_csv("Data/combined_ufc_dataset.csv")

# Load ILO scores dataset
ilo_df = pd.read_csv("Data/fighter_ilo_scores.csv")  # This file contains ILO scores

# Handle NaN values in 'REDWINS' column (fill NaN with 0)
df_combined['REDWINS'] = df_combined['REDWINS'].fillna(0)

# Merge ILO scores into the main dataset based on 'FIGHTER'
df_combined = pd.merge(df_combined, ilo_df[['FIGHTER', 'ILO']], on='FIGHTER', how='left')

# Step 1: Create new features
df_combined['KD_RATE'] = df_combined['KD'] / df_combined['ROUND'].replace(0, 1)
df_combined['SIGSTR_ACC'] = df_combined['SIGSTR_MADE'] / df_combined['SIGSTR_ATTEMPTED'].replace(0, 1)
df_combined['TD_ACC'] = df_combined['TD_MADE'] / df_combined['TD_ATTEMPTED'].replace(0, 1)
df_combined['FINISH_RATIO'] = (df_combined['SUBATT'] + df_combined['KD']) / df_combined['ROUND'].replace(0, 1)

# Step 2: Define target variable as multiclass (3 classes: Top-tier, Mid-tier, Low-tier)
df_combined['LABEL'] = pd.cut(df_combined['REDWINS'], bins=[0, 5, 10, 50], labels=['Low-tier', 'Mid-tier', 'Top-tier'])

# Ensure that 'LABEL' is a categorical variable
df_combined['LABEL'] = df_combined['LABEL'].astype('category')

# Step 3: Encode the LABEL column to numerical values
label_encoder = LabelEncoder()
df_combined['LABEL'] = label_encoder.fit_transform(df_combined['LABEL'])

# Step 4: Define features (including the ILO score)
selected_features = [
    'KD_RATE',
    'SIGSTR_ACC',
    'TD_ACC',
    'FINISH_RATIO',
    'CTRL',
    'REV',
    'REACHDIF',
    'HEIGHTDIF',
    'AGEDIF',
    'ILO'  # Include ILO score as a feature
]

X = df_combined[selected_features]
y = df_combined['LABEL']

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Set up the XGBoost model for multiclass classification
model = XGBClassifier(
    objective='multi:softmax',  # Specify multiclass classification
    num_class=3,  # Number of classes (adjust based on your target variable)
    use_label_encoder=False,
    eval_metric='mlogloss'
)

# Step 7: Train the model
model.fit(X_train, y_train)

# Step 8: Make predictions
y_pred = model.predict(X_test)

# Step 9: Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.93      0.78     34205
           1       0.80      0.57      0.67     22047
           2       0.86      0.65      0.74     14434
           3       0.97      0.49      0.65      7858

    accuracy                           0.73     78544
   macro avg       0.82      0.66      0.71     78544
weighted avg       0.77      0.73      0.73     78544

Confusion Matrix:
[[31918  1609   591    87]
 [ 8532 12575   906    34]
 [ 3954  1111  9359    10]
 [ 3539   356    90  3873]]


In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Load the main dataset (combined UFC dataset)
df_combined = pd.read_csv("Data/final_fighter_dataset.csv")  # Assuming the file name is correct

# Load ILO scores dataset
ilo_df = pd.read_csv("Data/fighter_ilo_scores.csv")  # This file contains ILO scores

# Handle NaN values in the 'Wins' column (fill NaN with 0)
df_combined['Wins'] = df_combined['Wins'].fillna(0)

# Merge ILO scores into the main dataset based on 'Fighter'
df_combined = pd.merge(df_combined, ilo_df[['FIGHTER', 'ILO']], left_on='Fighter', right_on='FIGHTER', how='left')

# Step 1: Create new features based on the available data
df_combined['KD_RATE'] = df_combined['KODif'] / df_combined['TotalRoundDif'].replace(0, 1)  # Adjust KD and rounds related columns
df_combined['SIGSTR_ACC'] = df_combined['RedAvgSigStrLanded'] / df_combined['RedAvgSigStrPct'].replace(0, 1)
df_combined['TD_ACC'] = df_combined['RedAvgTDLanded'] / df_combined['RedAvgTDPct'].replace(0, 1)
df_combined['FINISH_RATIO'] = (df_combined['RedWinsByKO'] + df_combined['RedWinsBySubmission']) / df_combined['RedTotalRoundsFought'].replace(0, 1)

# Step 2: Define target variable as multiclass (3 classes: Mid-tier, High-tier, Top-tier)
# Using the 'wins' column to define the target label
df_combined['LABEL'] = pd.cut(df_combined['Wins'], bins=[0, 10, 20, 50], labels=['Mid-tier', 'High-tier', 'Top-tier'])

# Ensure that 'LABEL' is a categorical variable
df_combined['LABEL'] = df_combined['LABEL'].astype('category')

# Step 3: Encode the LABEL column to numerical values
label_encoder = LabelEncoder()
df_combined['LABEL'] = label_encoder.fit_transform(df_combined['LABEL'])

# Step 4: Define features (including the ILO score)
selected_features = [
    'KD_RATE',
    'SIGSTR_ACC',
    'TD_ACC',
    'FINISH_RATIO',
    'RedHeightCms',  # Adjusting for the correct column name
    'RedReachCms',   # Adjusting for the correct column name
    'ReachDif',      # Including difference of reach
    'HeightDif',     # Including difference in height
    'ILO'  # Include ILO score as a feature
]

X = df_combined[selected_features]
y = df_combined['LABEL']

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Set up the XGBoost model for multiclass classification
model = XGBClassifier(
    objective='multi:softmax',  # Specify multiclass classification
    num_class=3,  # Number of classes (adjust based on your target variable)
    use_label_encoder=False,
    eval_metric='mlogloss'
)

# Step 7: Train the model
model.fit(X_train, y_train)

# Step 8: Make predictions
y_pred = model.predict(X_test)

# Add predictions to the dataset (you can add this column to the entire df_combined if needed)
df_combined['PREDICTED_LABEL'] = model.predict(X)

# Step 9: Save the updated DataFrame with the predicted labels to a new CSV file
df_combined.to_csv("Data/combined_ufc_dataset_with_predictions.csv", index=False)

# Step 10: Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.72      0.60       159
           1       0.45      0.29      0.36        92
           2       0.44      0.23      0.31        64

    accuracy                           0.50       315
   macro avg       0.47      0.41      0.42       315
weighted avg       0.48      0.50      0.47       315

Confusion Matrix:
[[114  31  14]
 [ 60  27   5]
 [ 47   2  15]]


In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Load the main dataset (combined UFC dataset)
df_combined = pd.read_csv("Data/final_fighter_dataset.csv")  # Assuming the file name is correct

# Load ILO scores dataset
ilo_df = pd.read_csv("Data/fighter_ilo_scores.csv")  # This file contains ILO scores

# Handle NaN values in the 'Wins' and 'Losses' columns (fill NaN with 0)
df_combined['Wins'] = df_combined['Wins'].fillna(0)
df_combined['Losses'] = df_combined['Losses'].fillna(0)

# Calculate the win-to-loss ratio, with a safeguard against division by zero
df_combined['Win_Loss_Ratio'] = df_combined['Wins'] / df_combined['Losses'].replace(0, 1)  # Replace 0 losses with 1 to avoid division by zero

# Merge ILO scores into the main dataset based on 'Fighter'
df_combined = pd.merge(df_combined, ilo_df[['FIGHTER', 'ILO']], left_on='Fighter', right_on='FIGHTER', how='left')

# Step 1: Create new features based on the available data
df_combined['KD_RATE'] = df_combined['KODif'] / df_combined['TotalRoundDif'].replace(0, 1)  # Adjust KD and rounds related columns
df_combined['SIGSTR_ACC'] = df_combined['RedAvgSigStrLanded'] / df_combined['RedAvgSigStrPct'].replace(0, 1)
df_combined['TD_ACC'] = df_combined['RedAvgTDLanded'] / df_combined['RedAvgTDPct'].replace(0, 1)
df_combined['FINISH_RATIO'] = (df_combined['RedWinsByKO'] + df_combined['RedWinsBySubmission']) / df_combined['RedTotalRoundsFought'].replace(0, 1)

# Define the target variable based on win to loss ratio with custom bins
df_combined['LABEL'] = pd.cut(df_combined['Win_Loss_Ratio'], 
                               bins=[-float('inf'), 2, 3.5, float('inf')],  # Manually defined ranges
                               labels=['Low-tier', 'Mid-tier', 'Top-tier'])



# Ensure that 'LABEL' is a categorical variable
df_combined['LABEL'] = df_combined['LABEL'].astype('category')

# Step 3: Encode the LABEL column to numerical values
label_encoder = LabelEncoder()
df_combined['LABEL'] = label_encoder.fit_transform(df_combined['LABEL'])

# Step 4: Define features (including the ILO score)
selected_features = [
    'KD_RATE',
    'SIGSTR_ACC',
    'TD_ACC',
    'FINISH_RATIO',
    'RedHeightCms',  # Adjusting for the correct column name
    'RedReachCms',   # Adjusting for the correct column name
    'ReachDif',      # Including difference of reach
    'HeightDif',     # Including difference in height
    'ILO'  # Include ILO score as a feature
]

X = df_combined[selected_features]
y = df_combined['LABEL']

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Set up the XGBoost model for multiclass classification
model = XGBClassifier(
    objective='multi:softmax',  # Specify multiclass classification
    num_class=3,  # Number of classes (adjust based on your target variable)
    use_label_encoder=False,
    eval_metric='mlogloss'
)

# Step 7: Train the model
model.fit(X_train, y_train)

# Step 8: Make predictions
y_pred = model.predict(X_test)

# Add predictions to the dataset (you can add this column to the entire df_combined if needed)
df_combined['PREDICTED_LABEL'] = model.predict(X)

# Step 9: Save the updated DataFrame with the predicted labels to a new CSV file
df_combined.to_csv("Data/combined_ufc_dataset_with_predictions.csv", index=False)

# Step 10: Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.31      0.21      0.25        99
           1       0.43      0.55      0.48       130
           2       0.36      0.35      0.36        86

    accuracy                           0.39       315
   macro avg       0.37      0.37      0.36       315
weighted avg       0.37      0.39      0.38       315

Confusion Matrix:
[[21 52 26]
 [32 71 27]
 [14 42 30]]


In [93]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler  # For oversampling

# Load the main dataset (combined UFC dataset)
df_combined = pd.read_csv("Data/final_fighter_dataset - Copy.csv")  # Assuming the file name is correct

# Load ILO scores dataset
ilo_df = pd.read_csv("Data/fighter_ilo_scores.csv")  # This file contains ILO scores

# Handle NaN values in the 'Wins' and 'Losses' columns (fill NaN with 0)
df_combined['Wins'] = df_combined['Wins'].fillna(0)
df_combined['Losses'] = df_combined['Losses'].fillna(0)

# Calculate the win-to-loss ratio, with a safeguard against division by zero
df_combined['Win_Loss_Ratio'] = df_combined['Wins'] / df_combined['Losses'].replace(0, 1)  # Replace 0 losses with 1 to avoid division by zero

# Merge ILO scores into the main dataset based on 'Fighter'
df_combined = pd.merge(df_combined, ilo_df[['FIGHTER', 'ILO']], left_on='Fighter', right_on='FIGHTER', how='left')

# Step 1: Create new features based on the available data
df_combined['KD_RATE'] = df_combined['KODif'] / df_combined['TotalRoundDif'].replace(0, 1)  # Adjust KD and rounds related columns
df_combined['SIGSTR_ACC'] = df_combined['RedAvgSigStrLanded'] / df_combined['RedAvgSigStrPct'].replace(0, 1)
df_combined['TD_ACC'] = df_combined['RedAvgTDLanded'] / df_combined['RedAvgTDPct'].replace(0, 1)
df_combined['FINISH_RATIO'] = (df_combined['RedWinsByKO'] + df_combined['RedWinsBySubmission']) / df_combined['RedTotalRoundsFought'].replace(0, 1)

# Define the target variable based on win to loss ratio with custom bins
df_combined['LABEL'] = pd.cut(df_combined['Win_Loss_Ratio'], 
                               bins=[-float('inf'), 2, 3, float('inf')],  # Manually defined ranges
                               labels=['Low-tier', 'Mid-tier', 'Top-tier'])

# Ensure that 'LABEL' is a categorical variable
df_combined['LABEL'] = df_combined['LABEL'].astype('category')

# Step 3: Encode the LABEL column to numerical values
label_encoder = LabelEncoder()
df_combined['LABEL'] = label_encoder.fit_transform(df_combined['LABEL'])

# Step 4: Define features (including the ILO score)
selected_features = [
    'KD_RATE',
    'SIGSTR_ACC',
    'TD_ACC',
    'FINISH_RATIO',
    'RedHeightCms',  # Adjusting for the correct column name
    'RedReachCms',   # Adjusting for the correct column name
    'ReachDif',      # Including difference of reach
    'HeightDif',     # Including difference in height
    'ILO'  # Include ILO score as a feature
]

X = df_combined[selected_features]
y = df_combined['LABEL']

# Step 5: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 6: Apply RandomOverSampler to balance the dataset (this will oversample the minority classes)
ros = RandomOverSampler(random_state=42)
X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

# Step 7: Set up the XGBoost model for multiclass classification
model = XGBClassifier(
    objective='multi:softmax',  # Specify multiclass classification
    num_class=3,  # Number of classes (adjust based on your target variable)
    use_label_encoder=False,
    eval_metric='mlogloss'
)

# Step 8: Train the model with the resampled data
model.fit(X_train_res, y_train_res)

# Step 9: Make predictions
y_pred = model.predict(X_test)

# Add predictions to the dataset (you can add this column to the entire df_combined if needed)
df_combined['PREDICTED_LABEL'] = model.predict(X)

# Step 10: Save the updated DataFrame with the predicted labels to a new CSV file
df_combined.to_csv("Data/combined_ufc_dataset_with_predictions.csv", index=False)

# Step 11: Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.32      0.28      0.30       138
           1       0.35      0.39      0.37       150
           2       0.38      0.38      0.38       155

    accuracy                           0.35       443
   macro avg       0.35      0.35      0.35       443
weighted avg       0.35      0.35      0.35       443

Confusion Matrix:
[[39 53 46]
 [41 58 51]
 [41 55 59]]


In [82]:
import joblib

# Save the trained model to a file
joblib.dump(model, 'ufc_fighter_model.pkl')

['ufc_fighter_model.pkl']

In [92]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder

# Load the main dataset (final fighter dataset with all features)
df_combined = pd.read_csv("Data/final_fighter_dataset.csv")  # Adjust path if needed

# Load ILO scores dataset
ilo_df = pd.read_csv("Data/fighter_ilo_scores.csv")  # This file contains ILO scores

# Handle NaN values in 'Wins' and 'Losses' columns
df_combined['Wins'] = df_combined['Wins'].fillna(0)
df_combined['Losses'] = df_combined['Losses'].fillna(0)

# Calculate the win-to-loss ratio, with a safeguard against division by zero
df_combined['Win_Loss_Ratio'] = df_combined['Wins'] / df_combined['Losses'].replace(0, 1)  # Avoid division by zero

# Merge ILO scores into the main dataset based on 'Fighter'
df_combined = pd.merge(df_combined, ilo_df[['FIGHTER', 'ILO']], left_on='Fighter', right_on='FIGHTER', how='left')

# Step 1: Create new features based on the available data
df_combined['KD_RATE'] = df_combined['KODif'] / df_combined['TotalRoundDif'].replace(0, 1)  # Adjust KD and rounds related columns
df_combined['SIGSTR_ACC'] = df_combined['RedAvgSigStrLanded'] / df_combined['RedAvgSigStrPct'].replace(0, 1)
df_combined['TD_ACC'] = df_combined['RedAvgTDLanded'] / df_combined['RedAvgTDPct'].replace(0, 1)
df_combined['FINISH_RATIO'] = (df_combined['RedWinsByKO'] + df_combined['RedWinsBySubmission']) / df_combined['RedTotalRoundsFought'].replace(0, 1)

# Define the target variable based on win to loss ratio with custom bins
df_combined['LABEL'] = pd.cut(df_combined['Win_Loss_Ratio'], 
                               bins=[-float('inf'), 2, 3, float('inf')],  # Manually defined ranges for Low, Mid, Top-tier
                               labels=['Low-tier', 'Mid-tier', 'Top-tier'])

# Ensure that 'LABEL' is a categorical variable
df_combined['LABEL'] = df_combined['LABEL'].astype('category')

# Step 3: Encode the LABEL column to numerical values
label_encoder = LabelEncoder()
df_combined['LABEL'] = label_encoder.fit_transform(df_combined['LABEL'])

# Step 4: Define features (including the ILO score)
selected_features = [
    'KD_RATE',
    'SIGSTR_ACC',
    'TD_ACC',
    'FINISH_RATIO',
    'RedHeightCms',  # Adjusting for the correct column name
    'RedReachCms',   # Adjusting for the correct column name
    'ReachDif',      # Including difference of reach
    'HeightDif',     # Including difference in height
    'ILO'  # Include ILO score as a feature
]

X = df_combined[selected_features]
y = df_combined['LABEL']

# Step 5: Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 6: Apply RandomOverSampler to balance the dataset (this will oversample the minority classes)
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

# Step 7: Set up the XGBoost model for multiclass classification
from xgboost import XGBClassifier

model = XGBClassifier(
    objective='multi:softmax',  # Specify multiclass classification
    num_class=3,  # Number of classes (adjust based on your target variable)
    use_label_encoder=False,
    eval_metric='mlogloss'
)

# Step 8: Train the model with the resampled data
model.fit(X_train_res, y_train_res)

# Step 9: Save the trained model
joblib.dump(model, 'ufc_fighter_model.pkl')

# Step 10: Now load a new dataset with 100 fighters (sample dataset that was not used in training)
new_fighters = pd.read_csv('Data/prediction_sample.csv')

# Step 11: Preprocess the new dataset (create the same features as in training)
# Merge ILO scores into the new fighters dataset
new_fighters = pd.merge(new_fighters, ilo_df[['FIGHTER', 'ILO']], left_on='Fighter', right_on='FIGHTER', how='left')

new_fighters['KD_RATE'] = new_fighters['KODif'] / new_fighters['TotalRoundDif'].replace(0, 1)  # Adjust KD and rounds related columns
new_fighters['SIGSTR_ACC'] = new_fighters['RedAvgSigStrLanded'] / new_fighters['RedAvgSigStrPct'].replace(0, 1)
new_fighters['TD_ACC'] = new_fighters['RedAvgTDLanded'] / new_fighters['RedAvgTDPct'].replace(0, 1)
new_fighters['FINISH_RATIO'] = (new_fighters['RedWinsByKO'] + new_fighters['RedWinsBySubmission']) / new_fighters['RedTotalRoundsFought'].replace(0, 1)

# Step 12: Make predictions on the new fighters
X_new = new_fighters[selected_features]
predictions = model.predict(X_new)

# Map predictions back to the original class names (Low-tier, Mid-tier, Top-tier)
new_fighters['Predicted_Label'] = label_encoder.inverse_transform(predictions)

# Display the results (Fighter Name and Predicted Label)
print(new_fighters[['Fighter', 'Predicted_Label']])

# Step 13: Save the predictions to a CSV file
new_fighters.to_csv('Data/fighters_with_predictions.csv', index=False)


           Fighter Predicted_Label
0       Aalon Cruz        Low-tier
1   Aaron Phillips        Mid-tier
2      Aaron Riley        Mid-tier
3       Aaron Rosa        Mid-tier
4    Aaron Simpson        Mid-tier
..             ...             ...
95       Andy Ogle        Low-tier
96      Ange Loosa        Top-tier
97   Angel Pacheco        Top-tier
98     Angela Hill        Low-tier
99   Angela Magana        Mid-tier

[100 rows x 2 columns]


In [101]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler  # For oversampling

# Load the necessary datasets
df_combined = pd.read_csv("Data/final_fighter_dataset.csv")  # Adjust path if needed
ilo_df = pd.read_csv("Data/fighter_ilo_scores.csv")  # This file contains ILO scores

# Handle NaN values in 'Wins' and 'Losses' columns
df_combined['Wins'] = df_combined['Wins'].fillna(0)
df_combined['Losses'] = df_combined['Losses'].fillna(0)

# Calculate the win-to-loss ratio, with a safeguard against division by zero
df_combined['Win_Loss_Ratio'] = df_combined['Wins'] / df_combined['Losses'].replace(0, 1)  # Avoid division by zero

# Merge ILO scores into the main dataset based on 'Fighter'
df_combined = pd.merge(df_combined, ilo_df[['FIGHTER', 'ILO']], left_on='Fighter', right_on='FIGHTER', how='left')

# Step 1: Feature Engineering for both fighters (Fighter 1 and Fighter 2)
df_combined['KD_RATE'] = df_combined['KODif'] / df_combined['TotalRoundDif'].replace(0, 1)  # Adjust KD and rounds related columns
df_combined['SIGSTR_ACC'] = df_combined['RedAvgSigStrLanded'] / df_combined['RedAvgSigStrPct'].replace(0, 1)
df_combined['TD_ACC'] = df_combined['RedAvgTDLanded'] / df_combined['RedAvgTDPct'].replace(0, 1)
df_combined['FINISH_RATIO'] = (df_combined['RedWinsByKO'] + df_combined['RedWinsBySubmission']) / df_combined['RedTotalRoundsFought'].replace(0, 1)

# Define the target variable based on win to loss ratio and fighter performance
df_combined['OUTCOME'] = df_combined.apply(
    lambda row: 1 if row['Win_Loss_Ratio'] > 2 and row['RedWins'] > row['BlueWins'] else 
    (2 if abs(row['Win_Loss_Ratio'] - row['RedWins']) < 1 and abs(row['Win_Loss_Ratio'] - row['BlueWins']) < 1 else 0), axis=1
)

# Ensure that 'OUTCOME' is a categorical variable
df_combined['OUTCOME'] = df_combined['OUTCOME'].astype('category')

# Step 2: Prepare features (using stats for both fighters)
# This assumes Fighter 1's data is in columns starting with "Red" and Fighter 2's data in columns starting with "Blue"
selected_features = [
    'KD_RATE',
    'SIGSTR_ACC',
    'TD_ACC',
    'FINISH_RATIO',
    'RedHeightCms',  # Adjusting for the correct column name
    'RedReachCms',   # Adjusting for the correct column name
    'ReachDif',      # Including difference of reach
    'HeightDif',     # Including difference in height
    'ILO'  # Include ILO score as a feature
]

# Combine the stats for Fighter 1 and Fighter 2
X = df_combined[['Red' + col for col in selected_features]].join(df_combined[['Blue' + col for col in selected_features]])

# Step 3: Encode the target variable (OUTCOME)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_combined['OUTCOME'])

# Step 4: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 5: Apply RandomOverSampler to balance the dataset
ros = RandomOverSampler(random_state=42)
X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

# Step 6: Set up the XGBoost model
model = XGBClassifier(
    objective='multi:softmax',
    num_class=3,  # Three classes: Win, Loss, Draw
    use_label_encoder=False,
    eval_metric='mlogloss'
)

# Step 7: Train the model with the resampled data
model.fit(X_train_res, y_train_res)

# Step 8: Save the trained model
import joblib
joblib.dump(model, 'ufc_fight_prediction_model.pkl')

# Step 9: Evaluate the model
y_pred = model.predict(X_test)

# Step 10: Evaluate the model's performance
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


KeyError: "None of [Index(['RedKD_RATE', 'RedSIGSTR_ACC', 'RedTD_ACC', 'RedFINISH_RATIO',\n       'RedRedHeightCms', 'RedRedReachCms', 'RedReachDif', 'RedHeightDif',\n       'RedILO'],\n      dtype='object')] are in the [columns]"

In [96]:
# Check the first few rows of the DataFrame to verify the structure
print(df_combined.head())


          Fighter   RedOdds  BlueOdds  RedExpectedValue  BlueExpectedValue  \
0      Aalon Cruz  0.664348  0.593478          0.062149           0.143511   
1  Aaron Phillips  0.783478  0.446739          0.191808           0.046022   
2     Aaron Riley  0.733913  0.506522          0.169088           0.089313   
3      Aaron Rosa  0.688696  0.569565          0.102009           0.093130   
4   Aaron Simpson  0.706087  0.543043          0.106848           0.152744   

   NumberOfRounds  BlueCurrentLoseStreak  BlueCurrentWinStreak  BlueDraws  \
0             0.0                  0.000              0.000000        0.0   
1             0.0                  0.125              0.000000        0.0   
2             0.0                  0.250              0.000000        0.0   
3             0.0                  0.000              0.000000        0.0   
4             0.0                  0.200              0.066667        0.0   

   BlueAvgSigStrLanded  ...  BlueDecOdds  RSubOdds  BSubOdds   RKOOd